<a href="https://colab.research.google.com/github/EVA4-RS-Group/Phase2/blob/master/S9/nlp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchtext import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy')
LABEL = data.LabelField(dtype = torch.float)

In [2]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:07<00:00, 11.0MB/s]


In [3]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [4]:
print(vars(train_data.examples[0]))

{'text': ['The', 'Sunshine', 'Boys', 'is', 'a', 'terrific', 'comedy', 'about', 'two', 'ex', '-', 'vaudevillians', 'who', 'reluctantly', 'reunite', 'for', 'a', 'TV', 'special', 'despite', 'the', 'fact', 'that', 'they', 'despise', 'each', 'other.<br', '/><br', '/>The', 'comic', 'genius', 'of', 'two', 'masters', 'at', 'work', ',', 'George', 'Burns', 'and', 'Walter', 'Matthau', 'are', 'stellar', '!', 'Some', 'of', 'the', 'best', 'scenes', 'are', 'when', 'the', 'duo', 'is', 'fighting', 'over', 'the', 'silliest', 'little', 'trivial', 'things', '!', 'The', 'material', 'is', 'fast', '-', 'paced', 'and', 'witty', ',', 'appealing', 'to', 'all', 'ages.<br', '/><br', '/>MILD', 'SPOILER', 'ALERT', ':', 'There', 'are', 'some', 'mildly', 'sad', 'moments', 'toward', 'the', 'end', 'of', 'the', 'movie', 'that', 'deal', 'indirectly', 'with', 'the', 'affects', 'of', 'aging', 'that', 'gives', 'the', 'film', 'a', 'soft', ',', 'sincere', ',', 'tenderness', 'that', 'shows', 'to', 'this', 'reviewer', 'that', '

In [5]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [6]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')


Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [7]:

MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [8]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [9]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 202206), (',', 191956), ('.', 166163), ('and', 109262), ('a', 109159), ('of', 100423), ('to', 93811), ('is', 76499), ('in', 61125), ('I', 54411), ('it', 53484), ('that', 49281), ('"', 43937), ("'s", 43367), ('this', 42560), ('-', 36717), ('/><br', 35639), ('was', 35257), ('as', 30425), ('with', 29991)]


In [10]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [11]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f8e2a3f2ae8>, {'neg': 0, 'pos': 1})


In [12]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [13]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [14]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [15]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,592,105 trainable parameters


In [16]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [17]:
criterion = nn.BCEWithLogitsLoss()

In [18]:
model = model.to(device)
criterion = criterion.to(device)

In [19]:

def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [20]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [21]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    count = 0
    with torch.no_grad():
    
        for batch in iterator:
            if (count <5):
              print(batch.text)
              count+=1
            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [22]:

import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [23]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

tensor([[   66,    11,    11,  ...,     0,   150,    11],
        [ 3441,    63,   215,  ...,     4,   846,    63],
        [   17,   142,    16,  ...,   246,     7,    27],
        ...,
        [   42,     2, 19274,  ...,     1,     1,     1],
        [ 2434,   425,    14,  ...,     1,     1,     1],
        [   40,    88,     4,  ...,     1,     1,     1]], device='cuda:0')
tensor([[   11,    11,    25,  ...,    11,    11,    66],
        [   92,    19, 11155,  ...,   242,   242,    24],
        [   75,  1523,   469,  ...,    16,    14, 21368],
        ...,
        [  578,    62,  1836,  ...,     1,     1,     1],
        [   22,    96,     4,  ...,     1,     1,     1],
        [    4,     4,     1,  ...,     1,     1,     1]], device='cuda:0')
tensor([[  66,   25, 1365,  ...,   11, 3992,  935],
        [  19,    0,  349,  ...,  278, 2267,  115],
        [   6,   19,  222,  ...,   16,   39,  133],
        ...,
        [ 800,   16,   80,  ...,    1,    1,    1],
        [  24,   38, 

In [24]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

tensor([[  11,  262,   11,  ..., 6571, 7720,    0],
        [  73,    6,  160,  ...,    5,    2,    0],
        [   8, 4777,  124,  ...,  782,  292,    0],
        ...,
        [  78,    4, 2201,  ...,    1,    1,    1],
        [ 654,    0,  419,  ...,    1,    1,    1],
        [   4,   40,    4,  ...,    1,    1,    1]], device='cuda:0')
tensor([[  337,    66,    11,  ...,   161,  4189,   487],
        [16084,    22,    57,  ...,    83,   702,     3],
        [  234,    19,    27,  ...,   704,    13,    16],
        ...,
        [12820,  5281,    16,  ...,     1,     1,     1],
        [    4,  1492,    38,  ...,     1,     1,     1],
        [   14,    40,     4,  ...,     1,     1,     1]], device='cuda:0')
tensor([[11860,    12,    66,  ...,   487,     0,    25],
        [   24,    74,   234,  ...,     3,     6,   288],
        [   13,     9,    36,  ...,    11,     0,    17],
        ...,
        [   23,   919,   883,  ...,     1,     1,     1],
        [  691,  1488,   894,  ..